In [10]:
import cantera as ct
import numpy as np

gas = ct.Solution('JetSurf_1_0.yaml')
gas.TP = 1000.0, 10 * ct.one_atm
gas.set_equivalence_ratio(phi=1.0, fuel="NC7H16", oxidizer={"O2": 1.0, "N2": 3.76})
sparse_weight = np.load('sparse_weight.npy', allow_pickle=True)
reduced_kenetics_index = np.where(sparse_weight > 9e-1)[0]

species_in_selected_reactions = set([])
for i in reduced_kenetics_index:
    species_in_selected_reactions.update(gas.reaction(i).reactants.keys())
    species_in_selected_reactions.update(gas.reaction(i).products.keys())
species_list = list(species_in_selected_reactions) + ['AR', 'HE', 'N2']
species = []
for sp in gas.species():
    if sp.name in species_list:
        species.append(sp)

species_idx_list = []
for sp in gas.species():
    if sp.name in species_list:
        species_idx_list.append(gas.species_index(sp.name))

reactions = []
for rct in gas.reactions():
    reac_indices = set(gas.species_index(species) for species in rct.reactants)
    prod_indices = set(gas.species_index(species) for species in rct.products)

    # Check if all reactants and products are exclusively from the selected species
    if reac_indices.issubset(set(species_idx_list)) and prod_indices.issubset(set(species_idx_list)):
        reactions.append(rct)

print(f"Number of species : {len(species)}.")
print(f"Number of reactions : {len(reactions)}.")

print(len(species), len(reactions))

reduced_gas = ct.Solution(
    thermo='ideal-gas', 
    kinetics='gas',
    species=species, 
    reactions=reactions,
)

from soln2yaml import write
write(reduced_gas, 'JetSurf_1_0.yaml', 'reduced_sl_49.yaml')

Number of species : 49.
Number of reactions : 330.
49 330


ImportError: attempted relative import with no known parent package